# Exploratory Data Analysis of Heavy Truck J1939 data

Ce notebook va servir de base dans l'analyse des données du `Heavy Truck`. Ces données reprennent une payload que nous pouvons retrouver avec le protocole **bus CAN J1939**.

**Payload d'un message sous protocole J1939:**

<p align="center">
    <img src="./images/j1939-payload.png" alt="Payload J1939" width="50%">
</p>

:warning: Attention à bien écouter Vendredi 26 la présentation de `Simon Bellemare` sur les données **bus CAN J1939** !

# Import libraries

In [ ]:
import plotly.express as px

from canlock.db.database import get_session, init_db
from canlock.decoder import SessionDecoder

# Comment obtenir les signaux d'une session ?

In [ ]:
# Aussi simple que ça !
init_db()

with get_session() as session:
    decoder = SessionDecoder(db=session)
    sessions = decoder.list_sessions()

df_session = decoder.decode(session_id=sessions[12].id)
df_session.set_index("timestamp", inplace=True)

In [ ]:
df_session

In [ ]:
px.line(df_session["ENGINE_SPEED"].dropna().reset_index(drop=False), x="timestamp", y="ENGINE_SPEED", title="Engine speed during the session")